In [3]:
# Imports (consolidated in the first cell)
import os
import pandas as pd
import pandas_gbq
from google.cloud import bigquery
import sqlite3
import zipfile

# Set up BigQuery client and authentication
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'C:\Users\jochoa\Documents\GitHub\wedge_project\service_account_key.json'
client = bigquery.Client()

# Directory where data files are stored
data_dir = r'C:\Users\jochoa\Documents\GitHub\wedge_project\data'

# Task 1: Data Upload to BigQuery
def check_data():
    """
    Queries the BigQuery table to count records by `card_no`
    for transactions where `card_no` is not equal to 3.
    """
    query = """
    SELECT card_no, COUNT(*) as count
    FROM `umt-msba.wedge_project_ochoa.transactions`
    WHERE card_no != 3
    GROUP BY card_no
    LIMIT 10
    """
    job = client.query(query)
    results = job.result()
    for row in results:
        print(f"Card No: {row.card_no}, Count: {row.count}")

def get_owner_records(limit=5):
    """
    Fetches records for owners (excluding card_no == 3) from BigQuery.
    """
    query = """
    SELECT datetime, register_no, emp_no, trans_no, upc, description, trans_type, card_no
    FROM `umt-msba.wedge_project_ochoa.transactions`
    WHERE card_no != 3
    LIMIT @limit
    """
    job_config = bigquery.QueryJobConfig(
        query_parameters=[bigquery.ScalarQueryParameter("limit", "INT64", limit)]
    )
    job = client.query(query, job_config=job_config)
    rows = [dict(row) for row in job.result()]
    return pd.DataFrame(rows)



In [4]:

# Use raw string for file path
file_path = r'C:\Users\jochoa\Documents\GitHub\wedge_project\data\transArchive_201609_clean.csv'

# Read the CSV file, correcting escape character issues
df = pd.read_csv(file_path, delimiter=',', na_values=['NULL', '\\N', r'\\N'])

# Display the first few rows of the dataframe to inspect it
df.head()


datetime  register_no  emp_no  trans_no  upc  description  \
0  2016-09-01 07:13:09           51      94         6    0       Change   
1  2016-09-01 08:28:43           51      94        37    0  Credit Card   
2  2016-09-01 08:44:56           51      94        47  TAX          Tax   
3  2016-09-01 07:20:34           51      94        10    0       Change   
4  2016-09-01 08:57:38           51      94        50    0  Credit Card   

  trans_type trans_subtype trans_status  department  ...  batchHeaderID  \
0          T            CA          NaN           0  ...            NaN   
1          T            CC          NaN           0  ...            NaN   
2          A           NaN          NaN           0  ...            NaN   
3          T            CA          NaN           0  ...            NaN   
4          T            CC          NaN           0  ...            NaN   

   local  organic  display  receipt  card_no  store  branch  match_id  \
0    0.0      NaN      NaN      0.0    20074      1       3       0.0   
1    0.0      NaN      NaN      0.0        3      1       3       0.0   
2    0.0      NaN      NaN      0.0    10499      1       3       0.0   
3    0.0      NaN      NaN      0.0    12539      1       3       0.0   
4    0.0      NaN      NaN      0.0    16360      1       3       0.0   

   trans_id  
0         7  
1         7  
2        12  
3         9  
4        11  

[5 rows x 50 columns]

In [ ]:
# Function to upload a DataFrame to BigQuery and append to a single table
def upload_to_bigquery(df, table_name="transactions"):
    table_id = f"wedge_project_ochoa.{table_name}" 
   
    
    # Upload the DataFrame to BigQuery using pandas_gbq and append to the table
    pandas_gbq.to_gbq(
        df, 
        table_id, 
        project_id="umt-msba", 
        if_exists="append", 
        api_method="load_csv"
    )

    print(f"Uploaded {len(df)} rows to {table_id}")

# Loop through all CSV files in the directory and upload each to BigQuery
for file_name in os.listdir(data_dir):
    if file_name.endswith('.csv'): 
        file_path = os.path.join(data_dir, file_name)
        print(f"Processing file: {file_path}")
        
        # Read the CSV into a DataFrame
        df = pd.read_csv(file_path, delimiter=',', na_values=['NULL', '\\N'])
        
        # Upload the DataFrame to the 'transactions' table in BigQuery
        upload_to_bigquery(df, table_name="transactions")


Processing file: C:\Users\jochoa\Documents\GitHub\wedge_project\data\transArchive_201001_201003_clean.csv


1it [03:03, 183.22s/it]


Uploaded 2998330 rows to wedge_project_ochoa.transactions
Processing file: C:\Users\jochoa\Documents\GitHub\wedge_project\data\transArchive_201004_201006_clean.csv


1it [03:02, 183.00s/it]


Uploaded 3185807 rows to wedge_project_ochoa.transactions
Processing file: C:\Users\jochoa\Documents\GitHub\wedge_project\data\transArchive_201007_201009_clean.csv


...

Uploaded 925314 rows to wedge_project_ochoa.transactions
Processing file: C:\Users\jochoa\Documents\GitHub\wedge_project\data\transArchive_201612_clean.csv


1it [01:12, 72.41s/it]


Uploaded 915707 rows to wedge_project_ochoa.transactions
Processing file: C:\Users\jochoa\Documents\GitHub\wedge_project\data\transArchive_201701_clean.csv


1it [01:12, 72.19s/it]

Uploaded 936741 rows to wedge_project_ochoa.transactions


In [5]:
## Task 2 

# Set up BigQuery authentication
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'C:\Users\jochoa\Documents\GitHub\wedge_project\service_account_key.json'

# Initialize BigQuery client
client = bigquery.Client()

# Check for the number of records with card_no not equal to 3
def check_data():
    query = """
    SELECT card_no, COUNT(*) as count
    FROM `umt-msba.wedge_project_ochoa.transactions`
    WHERE card_no != 3
    GROUP BY card_no
    LIMIT 10
    """
    job = client.query(query)
    
    # Get results
    results = job.result()
    
    # Print results to confirm that data exists
    for row in results:
        print(f"Card No: {row.card_no}, Count: {row.count}")


In [6]:
# Query for owner records (excluding card_no == 3)
def get_owner_records(limit=5):
    query = """
    SELECT datetime, register_no, emp_no, trans_no, upc, description, trans_type, card_no
    FROM `umt-msba.wedge_project_ochoa.transactions`
    WHERE card_no != 3
    LIMIT @limit
    """
    
    # Configuring query parameters
    job_config = bigquery.QueryJobConfig(
        query_parameters=[
            bigquery.ScalarQueryParameter("limit", "INT64", limit)
        ]
    )

    # Running the query
    job = client.query(query, job_config=job_config)
    
    # Fetch the results directly without schema checks for now
    results = job.result()

    # Convert results to a DataFrame
    rows = [dict(row) for row in results]
    if not rows:
        raise ValueError("No rows fetched, check if your query conditions are correct.")

    df = pd.DataFrame(rows)
    print("First few rows of the DataFrame:", df.head())
    return df


In [7]:
# Task 2:

# Set up BigQuery authentication
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'C:\Users\jochoa\Documents\GitHub\wedge_project\service_account_key.json'

# Initialize BigQuery client
client = bigquery.Client()

# Step 1: Fetch a list of all owners excluding card_no == 3
def get_owners_list():
    query = """
    SELECT DISTINCT card_no
    FROM `umt-msba.wedge_project_ochoa.transactions`
    WHERE card_no != 3
    """
    results = client.query(query).result()
    
    # Convert the results to a DataFrame to work with the list of owners
    owners_df = pd.DataFrame([row.card_no for row in results], columns=['card_no'])
    return owners_df

# Step 2: Sample a subset of owners based on a sample size
def sample_owners(owners_df, sample_size):
    sampled_owners = owners_df.sample(n=sample_size, random_state=42)
    return sampled_owners

# Step 3: Extract all records associated with the sampled owners
def extract_records_for_sampled_owners(sampled_owners):
    # Convert the list of sampled owner IDs to a string for the SQL IN clause
    owner_ids = tuple(sampled_owners['card_no'].tolist())
    
    query = f"""
    SELECT *
    FROM `umt-msba.wedge_project_ochoa.transactions`
    WHERE card_no IN {owner_ids}
    """
    
    # Execute the query and store the results in a DataFrame
    results = client.query(query).result()
    records_df = pd.DataFrame([dict(row) for row in results])
    return records_df

# Step 4: Write the sampled records to a local text file
def write_sample_to_file(df, output_file):
    df.to_csv(output_file, sep='\t', index=False)

# Step 5: Dynamically adjust sample size to achieve 250 MB file size
def get_250mb_sample(owners_df, initial_sample_size=250, target_size_mb=250):
    sample_size = initial_sample_size
    file_path = "test_sample.txt"
    
    while True:
        # Sample owners and extract their records
        sampled_owners = sample_owners(owners_df, sample_size=sample_size)
        sampled_records = extract_records_for_sampled_owners(sampled_owners)
        
        # Write a test sample to file
        sampled_records.to_csv(file_path, sep='\t', index=False)
        file_size = os.path.getsize(file_path) / (1024 * 1024)  # Size in MB
        
        if file_size >= target_size_mb:
            print(f"Achieved target size with sample size {sample_size}: {file_size:.2f} MB")
            return sampled_records  # Return the DataFrame when target size is met
        
        # Increase the sample size if file is too small
        sample_size += 50
        print(f"Current file size: {file_size:.2f} MB, increasing sample size to {sample_size}")

# Workflow Function (No `if __name__ == "__main__"`)
def generate_250mb_sample(output_file='sampled_owners_250MB.txt'):
    # Step 1: Get all distinct owner IDs excluding card_no == 3
    owners = get_owners_list()
    
    # Step 2: Dynamically adjust sample size to achieve 250 MB
    sampled_records = get_250mb_sample(owners, initial_sample_size=250, target_size_mb=250)
    
    # Step 3: Write the final sample to a local text file
    write_sample_to_file(sampled_records, output_file=output_file)

    print(f"Successfully wrote {len(sampled_records)} records for sampled owners to '{output_file}'.")

# Call the main workflow directly
generate_250mb_sample()


Current file size: 96.98 MB, increasing sample size to 300
Current file size: 117.68 MB, increasing sample size to 350
Current file size: 133.91 MB, increasing sample size to 400
Current file size: 162.88 MB, increasing sample size to 450
Current file size: 183.22 MB, increasing sample size to 500
Current file size: 208.98 MB, increasing sample size to 550
Current file size: 223.95 MB, increasing sample size to 600
Current file size: 247.58 MB, increasing sample size to 650
Achieved target size with sample size 650: 263.69 MB
Successfully wrote 1454891 records for sampled owners to 'sampled_owners_250MB.txt'.


In [ ]:
## Task 3

# Set up BigQuery authentication
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'C:\Users\jochoa\Documents\GitHub\wedge_project\service_account_key.json'

# Initialize BigQuery client
client = bigquery.Client()

# Helper function to run a query and return a DataFrame
def query_to_dataframe(query):
    job = client.query(query)
    return job.result().to_dataframe()

# Summary table 1: Sales by date by hour
def create_sales_by_date_by_hour():
    query = """
    SELECT
        DATETIME_TRUNC(DATETIME(datetime), HOUR) AS datetime_hour,
        EXTRACT(DATE FROM DATETIME(datetime)) AS date,
        EXTRACT(HOUR FROM DATETIME(datetime)) AS hour,
        SUM(total) AS total_sales,  -- Using 'total' column as the total sales amount
        COUNT(trans_no) AS transactions,
        SUM(CASE WHEN trans_status = '' OR trans_status = ' ' THEN quantity ELSE 0 END) AS items
    FROM `umt-msba.wedge_project_ochoa.transactions`
    WHERE trans_status NOT IN ('R', 'V')
    GROUP BY datetime_hour, date, hour
    ORDER BY date, hour
    """
    return query_to_dataframe(query)

# Summary table 2: Sales by owner by year by month
def create_sales_by_owner_by_year_by_month():
    query = """
SELECT card_no,
                   EXTRACT(YEAR from datetime) as year,
                   EXTRACT(MONTH from datetime) as month,
                   ROUND(SUM(total),2) AS sales,
                   COUNT(distinct(
                      CONCAT(CAST(EXTRACT(DATE from datetime) AS STRING),
                             CAST(register_no AS STRING),
                             CAST(emp_no AS STRING),
                             CAST(trans_no AS STRING)))) as transactions,
                   SUM(CASE WHEN (trans_status = 'V' or trans_status = 'R') THEN -1 ELSE 1 END) as items
                   FROM `umt-msba.wedge_transactions.transArchive_*`
                     WHERE department != 0 and
                          department != 15 and
                         (trans_status IS NULL or 
                          trans_status = ' ' or 
                          trans_status = 'V' or 
                          trans_status = 'R') 
                    GROUP BY card_no, year, month
                    ORDER BY card_no, year, month    
    """
    return query_to_dataframe(query)

# Summary table 3: Sales by product description by year by month
def create_sales_by_product_by_year_by_month():
    query = """
    SELECT
        upc,
        description,
        department AS department_number,
        EXTRACT(YEAR FROM DATETIME(datetime)) AS year,
        EXTRACT(MONTH FROM DATETIME(datetime)) AS month,
        SUM(total) AS total_sales,  -- Using 'total' column as the total sales amount
        COUNT(trans_no) AS transactions,
        SUM(quantity) AS items
    FROM `umt-msba.wedge_project_ochoa.transactions`
    GROUP BY upc, description, department_number, year, month
    ORDER BY upc, year, month
    """
    return query_to_dataframe(query)

# Function to create SQLite database and populate it with summary tables
def build_sqlite_database(db_file='wedge_summary.db'):
    # Create a SQLite connection
    conn = sqlite3.connect(db_file)
    
    # Create 'sales_by_date_by_hour' table
    sales_by_date = create_sales_by_date_by_hour()
    sales_by_date.to_sql('sales_by_date_by_hour', conn, if_exists='replace', index=False)
    
    # Create 'sales_by_owner_by_year_by_month' table
    sales_by_owner = create_sales_by_owner_by_year_by_month()
    sales_by_owner.to_sql('sales_by_owner_by_year_by_month', conn, if_exists='replace', index=False)
    
    # Create 'sales_by_product_description_by_year_by_month' table
    sales_by_product = create_sales_by_product_by_year_by_month()
    sales_by_product.to_sql('sales_by_product_description_by_year_by_month', conn, if_exists='replace', index=False)
    
    # Commit changes and close the connection
    conn.commit()
    conn.close()
    print("SQLite database 'wedge_summary.db' created with all summary tables.")

# Run the main workflow
if __name__ == "__main__":
    build_sqlite_database()


C:\Users\jochoa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\google\cloud\bigquery\table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(
C:\Users\jochoa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\google\cloud\bigquery\table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(
C:\Users\jochoa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\google\cloud\bigquery\table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


SQLite database 'wedge_summary.db' created with all summary tables.


In [6]:
## Zip the File

import zipfile

# Define the path to your .db file
db_file = 'wedge_summary.db'
zip_file = 'wedge_summary.zip'

# Create a zip file
with zipfile.ZipFile(zip_file, 'w') as zipf:
    zipf.write(db_file)

print(f"Successfully created {zip_file}")


Successfully created wedge_summary.zip


In [9]:
import os
import zipfile

# Define the path to the original .db file and the new .zip file
db_file = 'wedge_summary.db'
zip_file = 'wedge_summary.zip'

# Compress the .db file into a .zip archive
with zipfile.ZipFile(zip_file, 'w') as zipf:
    zipf.write(db_file)

# Split the .zip file into smaller parts of 45 MB each
def split_file(file_path, part_size=45 * 1024 * 1024):  # 45 MB parts
    part_number = 1
    with open(file_path, 'rb') as f:
        while True:
            chunk = f.read(part_size)
            if not chunk:
                break
            part_file = f"{file_path}.part{str(part_number).zfill(2)}"
            with open(part_file, 'wb') as pf:
                pf.write(chunk)
            part_number += 1

# Call the split function
split_file(zip_file)
print("Successfully split the zip file into smaller parts.")


Successfully split the zip file into smaller parts.
